In [ ]:
import requests
import time
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

url = "https://api.ioda.inetintel.cc.gatech.edu/v2/entities/query"
start = time.time()
country_to_region = {}
lock = threading.Lock()  # to protect shared dict


def fetchCountries():
  params = {
      "entityType": "country"
  }

  headers = {
      "accept": "*/*"
  }
  response = requests.get(url, headers=headers, params=params)
  print(response.json())
  country_to_code = {country["name"]:country["code"] for country in response.json()["data"]}
  country_names = [country["name"] for country in response.json()["data"]]
  print(country_to_code)
  print(len(country_to_code))
  return country_to_code

country_to_code = fetchCountries()
error=[]

def fetch(country_name,country_code):
  params = {
      "entityType": "region",
      "relatedTo": f"country/{country_code}"
  }

  headers = {
      "accept": "*/*"
  }

  response = requests.get(url, headers=headers, params=params)
  if response.status_code != 200:
    error.append((response.status_code,country_name))

  data = response.json()
  region_names = [region["name"] for region in data["data"]]
  with lock:
    country_to_region[country_name] = region_names



#for country_code in country_to_code.values():
with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(fetch, country_name,country_code) for country_name, country_code in country_to_code.items()]

    for future in as_completed(futures):
        try:
            future.result()
        except Exception as e:
            print(f"An error occurred: {e}")


print(country_to_region)


In [ ]:
# create table database in notion for all countries

from notion_client import Client
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
from dotenv import load_dotenv


# Initialize Notion client
NOTION_TOKEN = os.getenv('NOTION_TOKEN')
#DATABASE_ID =  userdata.get('DATABASE_ID')

notion = Client(auth=NOTION_TOKEN)

# Load .env from the working directory
load_dotenv()

# Read MAIN_PAGE_ID from environment, fallback to colab userdata if not set
main_page_id = os.getenv("MAIN_PAGE_ID") 
created_database_ids = []


def clean_multiselect_value(value):
    return value.strip().title() if value else "Unknown"

def get_existing_databases(page_id: str):
    """
    Get all child databases on a given Notion page.

    Returns:
        dict: {database_title: database_id}
    """
    databases = {}
    has_more = True
    start_cursor = None

    while has_more:
        #print("===",page_id)
        page = notion.pages.retrieve(page_id=page_id)
        #print("-----",page)
        response = notion.blocks.children.list(
            block_id=page["id"],
            start_cursor=start_cursor
        )

        for block in response["results"]:
            if block["type"] == "child_database":
                title = block["child_database"]["title"].strip()
                databases[title] = block["id"]

        has_more = response.get("has_more", False)
        start_cursor = response.get("next_cursor")

    return databases



def create_database_link(country_name, region_list, existing_databases):
    try:
        if country_name in existing_databases:
            print(f"Database link for {country_name} already exists")
            return existing_databases[country_name]

        # Create database as a child of main page
        database = notion.databases.create(
            parent={"type": "page_id", "page_id": main_page_id},
            title=[{"type": "text", "text": {"content": country_name}}],
            properties={
                "Company / Data source": {"title": {}},
                "Region": {"rich_text": {}},
                "Description": {"rich_text": {}},
                "Comments on Web Scraping": {"rich_text": {}},
                "Link": {"rich_text": {}},
                "Scraping code created": {"rich_text": {}},
                "Challenges": {"multi_select": {}},
                "Data Type": {"multi_select": {}},
                "Scraping Frequency": {"multi_select": {}},
                "Status": {"multi_select": {}}
            }
        )
        db_id = database["id"]

        print(f"Created database for {country_name}")
        return db_id

    except Exception as e:
        print(f"Error creating database link for {country_name}: {e}")
        return None


# Get all existing databases on main page
existing_databases = get_existing_databases(main_page_id)

# Run concurrently
with ThreadPoolExecutor(max_workers=2) as executor:
    for cn, rl in country_to_region.items():
        futures.append(
            executor.submit(create_database_link, cn, rl, existing_databases)
          )
    for future in as_completed(futures):
        db_id = future.result()
        if db_id:
            created_database_ids.append(db_id)

print("Created database links:", created_database_ids)


In [ ]:
# prompt for web Scraping power outages company data by country and region_list

def build_prompt(country):
    return f"""You are a data scraper assistant.
Return a JSON array of all **power supply companies** data operating in {country} as a list.

For each company, strictly return JSON objects in this format:


[
  {{
    "Company / Data source": "Company Name",
    "Region": "Area in which the company operates",
    "Description": "What kind of power outage data the site provides (if no data, mention 'No outage data available')",
    "Comments on Web Scraping": "Step-by-step manual scraping method or 'Not applicable' if no data",
    "Power Outage Link": "URL of the official company page where outage data is (if none, use 'N/A'), Please provide the specific page not homepage",
    "Website": "URL of the homepage of the official company website",
    "Official Government Website URL": "URL of the official government website page where we got this company info",
    "Scraping code created": "no",
    "Challenges": "Challenges faced in scraping the data (or 'No challenges' if none)",
    "Data Type": "Future | Historical | Excel | None",
    "Scraping Frequency": "what is the frequency of data update? daily | weekly | monthly",
    "Status": "Not Started"
  }},
  ...
]
⚠️ Very Important:
- Include **all companies** in {country} even if they do not provide outage data.
- also scrape website if its in non english language.
- Return only a valid JSON array, no explanation text outside JSON.
    """

In [ ]:
# code for using above prompt to scrape data


from google.colab import userdata
from openai import OpenAI
import json

# client = OpenAI(
#     base_url="https://models.github.ai/inference",
#     api_key=userdata.get('OPENAI_KEY'),
# )
client = OpenAI(
    api_key=os.getenv('OPENAI_KEY'),
)
for country, _ in country_to_region.items():
    prompt = build_prompt(country)
    ASSISTANT = "You are a data scraper assistant. Your job is to search internet to get power outage data. Go to power companies page and provide correct url of power outage pages and data asked for."
    response = client.responses.create(
    model="gpt-5-mini",
    tools=[{"type": "web_search"}],
    input=prompt,
)
text = response.output[-1].content[0].text
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "given this chatgpt response api data. Extract scrapped json data from it.Only return json string which i can convert to python dict directly donot add any additional string"},
        {"role": "user", "content": response.output[-1].content[0].text},
    ],
);

#print( response)
# Get JSON output
#print(response.output[-1].content)
json_output = response.choices[0].message.content
#print(json_output)
data = json.loads(json_output)   # Python dict/list
#print(data)



In [ ]:
# use data from above web scraper and update corresponding notion table

import re
from notion_client import Client


NOTION_TOKEN =os.getenv('NOTION_TOKEN')
DATABASE_ID =  os.getenv('DATABASE_ID')

notion = Client(auth=NOTION_TOKEN)


def clean_multiselect_value(value: str) -> str:
    """Remove special characters not allowed in Notion multi_select values."""
    if not value:
        return "N/A"
    # Keep only alphanumerics, spaces, hyphens, and underscores
    cleaned = re.sub(r"[^A-Za-z0-9 _-]", "", value)
    return cleaned.strip()

def page_exists(company_name, region):
    query = notion.databases.query(
        database_id=DATABASE_ID,
        filter={
            "and": [
                {
                    "property": "Company / Data source",
                    "title": {"equals": company_name}
                },
                {
                    "property": "Region",
                    "rich_text": {"equals": region}
                }
            ]
        }
    )
    return len(query.get("results", [])) > 0


for company in data:   # loop over JSON list
    company_name = company["Company / Data source"]
    region = company["Region"]

    if page_exists(company_name, region):
        print(f"Skipping duplicate: {company_name} ({region})")
        continue
    notion.pages.create(
        parent={"database_id": DATABASE_ID},
        properties={
        "Company / Data source": {"title": [{"text": {"content": company["Company / Data source"]}}]},
        "Region": {"rich_text": [{"text": {"content": company["Region"]}}]},
        "Description": {"rich_text": [{"text": {"content": company["Description"]}}]},
        "Comments on Web Scraping": {"rich_text": [{"text": {"content": company["Comments on Web Scraping"]}}]},
        "Link": {"rich_text": [{"text": {"content": company["Link"]}}]},
        "Scraping code created": {"rich_text": [{"text": {"content": company["Scraping code created"]}}]},
        "Challenges": {"multi_select": [{"name": clean_multiselect_value(company["Challenges"])}]},
        "Data Type": {"multi_select": [{"name": clean_multiselect_value(dt.strip())} for dt in company["Data Type"].split("|") if dt.strip()]},
        "Scraping Frequency": {
            "multi_select": [
                {"name": clean_multiselect_value(company.get("Scraping Frequency", "Once").capitalize())}
            ]
        },
        "Status": {"multi_select": [{"name": clean_multiselect_value(company["Status"])}]}
        }
    )
